# Libraries 

In [1]:
import os
import glob
import subprocess
import json
import warnings
from typing import List, Union
from typing import Optional

import ee
import geemap
import rasterio
import fiona
from osgeo import gdal, osr, gdalconst

### Analysis environment 

#### Download and install anaconda:

https://www.anaconda.com/products/distribution

Accept the default installtion options

#### Setting up a conda environment:

#### Open Anaconda Navigator (search in Windows search bar), go to the 'Environment' tab and press 'Create'.

#### Call the new environment 'pygis'.

#### Select 'Python' in the packages section


After the environment is created, close the navigator and open Anaconda Prompt (search in Windows search bar)

Type in the console (wait for the downloads and installs, accept any prompt with 'Y' and press enter):

conda activate pygis


conda install -c conda-forge mamba

mamba install -c conda-forge pygis

conda install -c conda-forge jupyterlab jupyterlab-git

once installed, type jupyter-lab into the prompt to start the notebook

# EE log in 
Authenticate to Earth Engine with the unhashed cell below. Follow prompts and Google will provide a token to paste into the diologue box. Tokens last a week 

In [ ]:
#ee.Authenticate()

In [2]:
ee.Initialize()

# Functions

In [3]:
# Define a function to clip each image to the feature collection
def clip_image(image):
    """
    Clip an image to the specified feature collection.

    Args:
      image: The image to be clipped (ee.Image).
      site: The feature collection used for clipping (ee.FeatureCollection).

    Returns:
      The clipped image (ee.Image).
    """
    clipped_image = image.reproject(crs='EPSG:4326', scale=1000).clip(region)
    return clipped_image


def convert_modis_lst_to_degrees(lst_image: ee.Image) -> ee.Image:
    """
    Convert MODIS Terra Land Surface Temperature (LST) values to degrees Celsius.

    Args:
      lst_image: The MODIS Terra LST image (ee.Image).

    Returns:
      The MODIS Terra LST image converted to degrees Celsius (ee.Image).
    """
    # Scale the LST image by a factor of 0.02
    # Create a mask to retain only good quality data
    good_quality_mask = lst_image.select('QC_Day').bitwiseAnd(0b000000001111).eq(0)
    
    # Apply the mask to the LST image
    filtered_lst = lst_image.updateMask(good_quality_mask)
    # could add the filter above to below 
    scaled_lst = lst_image.select('LST_Day_1km').multiply(0.02)
    
    # Convert the scaled LST image to degrees Celsius
    lst_deg_c = scaled_lst.subtract(273.15).rename('LST')
    
    return lst_deg_c

def download_modis_lst(collection_id: str, output_dir: str, start_date: str, end_date: str, site: ee.FeatureCollection) -> None:
    """
    Download MODIS Terra Land Surface Temperature (LST) imagery.

    Args:
      collection_id: The Earth Engine collection ID for MODIS Terra LST (str).
      output_dir: The output directory to save the downloaded images (str).
      start_date: The start date of the time period (str, e.g., 'yyyy-mm-dd').
      end_date: The end date of the time period (str, e.g., 'yyyy-mm-dd').
      site: The feature collection used for filtering and clipping (ee.FeatureCollection).

    Returns:
      None
    """
    # Load the Earth Engine collection
    collection = (ee.ImageCollection(collection_id)
                    .filterBounds(site)
                    .map(clip_image)
                    .filterDate(start_date, end_date)
                    .map(convert_modis_lst_to_degrees)
                 )
    
    # Get the image IDs
    image_ids = collection.aggregate_array('system:index').getInfo()

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through the image IDs and download each image
    for image_id in image_ids:
        image = ee.Image(collection.filter(ee.Filter.eq('system:index', image_id)).first()).select(['LST'])
        # Get the image download URL
        geemap.ee_export_image(
               image,
               filename=os.path.join(output_dir, f'{image_id}.tif'),
               scale=1000,
               region=site.geometry(),
               file_per_band=False
            )
        print(f"Downloaded {image_id}.tif")

In [4]:
### upload shp to EE, this will automatically create a new feature in WGS84
region = geemap.shp_to_ee(os.path.join(os.getcwd(),
          'Data\\shp\\South_west_agricultural_region_DPIRD_008_gcs.shp'))

In [5]:
year = '2022'
start_date = year + '-01-01'
end_date = year + '-12-30'
output_dir = os.path.join(os.getcwd(), 'Data\\LST_Downloads\\'+year)
collection_id = "MODIS/061/MOD11A1"
download_modis_lst(collection_id, output_dir, start_date=start_date, end_date=end_date, site=region)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\MODIS-LST\Data\LST_Downloads\2022\2022_01_01.tif
Downloaded 2022_01_01.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\MODIS-LST\Data\LST_Downloads\2022\2022_01_02.tif
Downloaded 2022_01_02.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\MODIS-LST\Data\LST_Downloads\2022\2022_01_03.tif
Downloaded 2022_01_03.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\MODIS-LST\Data\LST_Downloads\2022\2022_01_04.tif
Downloaded 2022_01_04.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\MODIS-LST\Data\LST_Downloads\2022\2022_01_05.tif
Downloaded 2022_01_05.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\MODIS-LST\Data\LST_Downloads\2022\2022_01_06.tif
Downloaded 2022_01_06.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\00097030\Git\MODIS-LST\Data\LST